In [4]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/HISTORY BOM'
output_url = '../output/history_bom.xlsx'
file_names = next(walk(path), (None, None, []))[2]
file_names = [f'{path}/{x}' for x in file_names]

errors = []
for x in file_names:
    try:
        pd.read_excel(x, header=1)
    except KeyError:
        errors.append(x)

file_names = [x for x in file_names if x not in errors]
dfs = [pd.read_excel(x, header=1) for x in file_names]

errors = [(x.split('/')[-1]) for x in errors]

bom = pd.concat(dfs)

bom.to_excel('../output/HISTORY.xlsx')

bom = bom.loc[~pd.isna(bom['Line'])]

del bom['Unnamed: 0']

# Create level
bom['Line'] = bom['Line'].astype(str)
bom['Line'] = bom['Line'].str[-1]
bom['Line'] = pd.to_numeric(bom['Line'])

bom = bom[['Line', 'Component', 'Description', 'Specification' ,'Unit', 'Quantity']]

def find_if_item_is_child(row):
    if row['Line'] > 0:
        return True
bom['is_child'] = bom.apply(lambda row: find_if_item_is_child(row), axis=1)
def create_columns(row, i):
    if row['Line'] == i:
        return row['Component']
for i in bom['Line'].unique():
    bom[i] = bom.apply(lambda row: create_columns(row, i), axis=1)

bom.ffill(inplace=True)

def fill_father_name(row):
    if row['is_child'] and row['Line'] > 0:
        return row[row['Line'] - 1]

bom['father'] = bom.apply(lambda row: fill_father_name(row), axis=1)
bom['Product'] = bom[0]
for i in bom['Line'].unique():
    del bom[i]

bom.to_excel(output_url)


In [ ]:

# bom['temp'] = bom['Level'].shift()
# bom['temp'].fillna(0, inplace=True)

# # RETURN TRUE IF ITEM HAS FATHER
# def let_find_your_father(row):
#     if row['Level'] > row['temp']:
#         return True

# bom['is_child'] = bom.apply (lambda row: let_find_your_father(row), axis=1)
# del bom['temp']


# bom['temp_material'] = bom['Component'].shift()
# def fill_father_name(row):
#     if row['is_child']:
#         return row['temp_material']

# bom['father'] = bom.apply (lambda row: fill_father_name(row), axis=1)
    
# def fill_finish_product_name(row):
#     if row['Level'] == 0:
#         return row['Component']

# bom['product'] = bom.apply (lambda row: fill_finish_product_name(row), axis=1)
# del bom['temp_material']
# # bom.ffill(inplace=True)
# def finish_up(row):
#     if row['Level'] == 0:
#         return None
#     else:
#         return row['father']

# bom['father'] = bom.apply (lambda row: finish_up(row), axis=1)